## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

# Import API key
sys.path.insert(0,'..')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,62.01,96,100,4.21,overcast clouds
1,1,Vaini,TO,-21.2000,-175.2000,73.40,100,75,1.14,broken clouds
2,2,Vestmannaeyjar,IS,63.4427,-20.2734,45.09,71,20,11.81,few clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,68.52,77,4,15.52,clear sky
4,4,Cabo San Lucas,MX,22.8909,-109.9124,75.99,53,1,5.75,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip?"))
max_temp = float(input("What is your desired maximum temperature for your trip?"))

What is your desired minimum temperature for your trip?75
What is your desired maximum temperature for your trip?100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Cabo San Lucas,MX,22.8909,-109.9124,75.99,53,1,5.75,clear sky
5,5,Tikaitnagar,IN,26.9500,81.5833,94.01,10,14,3.22,few clouds
10,10,Avarua,CK,-21.2078,-159.7750,80.60,83,20,6.91,few clouds
12,12,Kieta,PG,-6.2167,155.6333,78.49,80,95,3.22,overcast clouds
15,15,Belmonte,BR,-15.8631,-38.8828,82.40,65,40,9.22,scattered clouds
16,16,Kanniyakumari,IN,8.0833,77.5667,83.82,78,45,13.67,scattered clouds
20,20,Rikitea,PF,-23.1203,-134.9692,76.03,71,19,9.24,few clouds
21,21,Lahij,YE,13.1667,44.5833,82.56,56,69,7.47,broken clouds
26,26,Butaritari,KI,3.0707,172.7902,82.44,76,100,9.15,overcast clouds
30,30,Mayo,US,38.8876,-76.5119,84.99,36,1,6.91,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                256
City                   256
Country                256
Lat                    256
Lng                    256
Max Temp               256
Humidity               256
Cloudiness             256
Wind Speed             256
Current Description    256
dtype: int64

In [6]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Cabo San Lucas,MX,75.99,clear sky,22.8909,-109.9124,
5,Tikaitnagar,IN,94.01,few clouds,26.9500,81.5833,
10,Avarua,CK,80.60,few clouds,-21.2078,-159.7750,
12,Kieta,PG,78.49,overcast clouds,-6.2167,155.6333,
15,Belmonte,BR,82.40,scattered clouds,-15.8631,-38.8828,
...,...,...,...,...,...,...,...
685,Dehloran,IR,83.75,clear sky,32.6941,47.2679,
691,Manzanillo,CU,93.16,clear sky,20.3433,-77.1167,
692,Siocon,PH,78.93,overcast clouds,7.7061,122.1353,
693,Sao Geraldo Do Araguaia,BR,84.96,overcast clouds,-6.4006,-48.5550,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found...skipping.")
        pass

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Cabo San Lucas,MX,75.99,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
5,Tikaitnagar,IN,94.01,few clouds,26.9500,81.5833,Suraj Verma S/O Om Prakash Verma
10,Avarua,CK,80.60,few clouds,-21.2078,-159.7750,Paradise Inn
12,Kieta,PG,78.49,overcast clouds,-6.2167,155.6333,Uruna Bay Retreat
15,Belmonte,BR,82.40,scattered clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
16,Kanniyakumari,IN,83.82,scattered clouds,8.0833,77.5667,Sparsa
20,Rikitea,PF,76.03,few clouds,-23.1203,-134.9692,Pension Maro'i
21,Lahij,YE,82.56,broken clouds,13.1667,44.5833,
26,Butaritari,KI,82.44,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
30,Mayo,US,84.99,clear sky,38.8876,-76.5119,Resorts Resource Group International


In [11]:
hotel_df.count()

City                   256
Country                256
Max Temp               256
Current Description    256
Lat                    256
Lng                    256
Hotel Name             256
dtype: int64

In [12]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [13]:
hotel_df.dtypes

City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how="any")

In [19]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Cabo San Lucas,MX,75.99,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
5,Tikaitnagar,IN,94.01,few clouds,26.9500,81.5833,Suraj Verma S/O Om Prakash Verma
10,Avarua,CK,80.60,few clouds,-21.2078,-159.7750,Paradise Inn
12,Kieta,PG,78.49,overcast clouds,-6.2167,155.6333,Uruna Bay Retreat
15,Belmonte,BR,82.40,scattered clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
16,Kanniyakumari,IN,83.82,scattered clouds,8.0833,77.5667,Sparsa
20,Rikitea,PF,76.03,few clouds,-23.1203,-134.9692,Pension Maro'i
21,Lahij,YE,82.56,broken clouds,13.1667,44.5833,
26,Butaritari,KI,82.44,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
30,Mayo,US,84.99,clear sky,38.8876,-76.5119,Resorts Resource Group International


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vaction.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Conditions</dt><dd>{Current Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))